# Ising Model using ITensor (Julia)

This is a personal notebook for myself, though hopefully it also helps others understand how some functions in ITensors work.
This is basically an editted notebook that uses the following link: itensor.org/docs.cgi?vers=julia&page=formulas/mixed_sites_dmrg


In this notebook, we will consider an ising model with $N$ sites. The Hamiltonian for the ising model is given as the following:

$$ H = -J\sum_{j=1}^{N-1} \sigma^z_{j} \sigma^z_{j+1} - h \sum_{j=1}^{N} \sigma^x_j $$

Note that the first sum is a two-body operator and the sum ends at $N-2$. This makes it so that the final pair is operating on sites $N-2, N-1$. The second term is a one-body operator. 


## Code

First, we need to import ITensors. (I include a print("done") line to make sure that the package is imported.)



In [2]:
using ITensors
print("done")

done

In the cell below, we construct the code for this Ising Model Calculation.

### Parameters

$N$ is the number of sites.

$Jzz$ is equivalent to $J$, and is the strength of the nearest neighbor interactions.

$Jx$ is equivalent to $h$, and is the strength of the term proportional to $\sigma^x$


### Breakdown

In the code cell below, we start by defining our three parameters. Next, we create our array of N spins. In the 'let' statement, we start by creating a collection of matrix product operator. Then, in the for loop, we iterate from $j=1$ to $N-1$ and to our MPO we add $\sigma^x_j$ and $\sigma^z_j\sigma^z_{j+1}$ operators with the given strenghts. Since the for loop terminates at $j=N-1$, at the end, we include an additional operator for the $N$th site, since the for loop does not capture the final $\sigma^x$ operator.

We then use the MPO() function to take the collection of operators and turn them into a formal Matrix Product Operator, with the proper indices and labels. We can think of this step as creating the Hamiltonian. The following lines then define accuracy parameters: the number of sweeps, maximum dimensions, bond length, and cutoffs.

psi0 is a random matrix product state. This will be our initial state that we will variationally modify to find the ground state energy. 

The final function, dmrg(), uses DMRG to calculate the ground state energy.


In [23]:
# Defining Parameters
N = 100 # number of sites
Jzz = 1.0 # Nearest-Neighbor interaction strength
Jx = 0.0 # transverse field interaction strength

# Building an Array of N spins
sites = siteinds("S=1/2", N)
# the S=1/2 is the label for the sites
# Note that we expect for Jzz = 1.0 and Jx = 0.0, for S=1/2 we should get E_{gs} = -99/(2*2) = -25.
# If we changed spin to 1, for example, with the same parameters, we would get -99/1*1 = -100

let
    ampo = AutoMPO()
    # AutoMPO() stores a collection of operator terms to be summed together into an MPO
    # Do so by calling the MPO function on the AutoMPO object
    # Which local op names are available is determined by the function op associated with the TagType
    # defiend by special index tags, like S=1/2, S=1, Fermion, electron
    
    # This segment below begins adding operators to our ampo.
    # We add in the order: Number, String, Int, String, Int
    # The ints are the site numbers. The strings are the names
    # the number is the coefficient
    for j=1:N-1 # iterates j=1, 2, ..., N-1 # there are N-1 pairs
        ampo += Jx, "Sx", j
        ampo += Jzz, "Sz", j, "Sz", j+1 #two body operator
    end
    ampo += Jx, "Sx", N

    H = MPO(ampo, sites) 
    # MPO() converts an AutoMPO object to an MPO with the indices given by sites. 
    
    sweeps = Sweeps(10)
    # The number of times we run the calculation
    
    # helps us define accuracy parameters
    maxdim!(sweeps, 10, 20, 100, 100, 200)
    # set max bond/link dimension for each sweep
    
    cutoff!(sweeps, 1E-10)
    # sets the truncation error cutoff for each sweep
    # Our sweeps is set to 10. We only have 9 params for maxdim and 1 for cutoff
    # if ferer params are given than the number of sweepsm then the last value is used
    # for all remaining sweeps.
    
    psi0 = randomMPS(sites, 2) # random MPS of bond dimension 2
    energy, psi = dmrg(H, psi0, sweeps)
    return
end


After sweep 1 energy=-24.749848318459 maxlinkdim=8 time=0.260
After sweep 2 energy=-24.750000000000 maxlinkdim=1 time=0.149
After sweep 3 energy=-24.750000000000 maxlinkdim=1 time=0.094
After sweep 4 energy=-24.750000000000 maxlinkdim=1 time=0.086
After sweep 5 energy=-24.750000000000 maxlinkdim=1 time=0.087
After sweep 6 energy=-24.750000000000 maxlinkdim=1 time=0.267
After sweep 7 energy=-24.750000000000 maxlinkdim=1 time=0.138
After sweep 8 energy=-24.750000000000 maxlinkdim=1 time=0.212
After sweep 9 energy=-24.750000000000 maxlinkdim=1 time=0.165
After sweep 10 energy=-24.750000000000 maxlinkdim=1 time=0.238
